Da biste pokrenuli sljedeće bilježnice, ako to još niste učinili, trebate postaviti openai ključ unutar .env datoteke kao `OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Sljedeće, učitat ćemo Indeks ugnježđivanja u Pandas DataFrame. Indeks ugnježđivanja pohranjen je u JSON datoteci pod nazivom `embedding_index_3m.json`. Indeks ugnježđivanja sadrži ugnježđivanja za svaki od YouTube transkripata do kraja listopada 2023. godine.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Zatim ćemo stvoriti funkciju nazvanu `get_videos` koja će pretraživati Indeks ugradnje za upit. Funkcija će vratiti top 5 videozapisa koji su najsličniji upitu. Funkcija radi na sljedeći način:

1. Prvo se stvara kopija Indeksa ugradnje.
2. Zatim se izračunava Ugradnja za upit koristeći OpenAI Embedding API.
3. Zatim se u Indeks ugradnje stvara novi stupac nazvan `similarity`. Stupac `similarity` sadrži kosinusnu sličnost između Ugradnje upita i Ugradnje za svaki segment videozapisa.
4. Zatim se Indeks ugradnje filtrira prema stupcu `similarity`. Indeks ugradnje se filtrira tako da uključuje samo videozapise koji imaju kosinusnu sličnost veću ili jednaku 0.75.
5. Na kraju, Indeks ugradnje se sortira prema stupcu `similarity` i vraća se top 5 videozapisa.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Ova funkcija je vrlo jednostavna, ona samo ispisuje rezultate pretraživanja.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Prvo se Ugrađeni Indeks učitava u Pandas Dataframe.
2. Zatim se korisnika traži da unese upit.
3. Nakon toga se poziva funkcija `get_videos` za pretraživanje Ugrađenog Indeksa prema upitu.
4. Na kraju se poziva funkcija `display_results` za prikaz rezultata korisniku.
5. Zatim se korisnika ponovo traži da unese drugi upit. Ovaj proces se nastavlja dok korisnik ne unese `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc.hr.png)

Bit ćete upitani da unesete upit. Unesite upit i pritisnite enter. Aplikacija će vratiti popis videozapisa koji su relevantni za upit. Aplikacija će također vratiti poveznicu na mjesto u videozapisu gdje se nalazi odgovor na pitanje.

Evo nekoliko upita koje možete isprobati:

- Što je Azure Machine Learning?
- Kako funkcioniraju konvolucijske neuronske mreže?
- Što je neuronska mreža?
- Mogu li koristiti Jupyter Notebooks s Azure Machine Learningom?
- Što je ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Odricanje od odgovornosti**:
Ovaj dokument je preveden korištenjem AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo postići točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati autoritativnim izvorom. Za kritične informacije preporučuje se profesionalni ljudski prijevod. Ne snosimo odgovornost za bilo kakva nesporazuma ili pogrešna tumačenja koja proizlaze iz korištenja ovog prijevoda.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
